##  Import Packages

In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import numpy as np
import os

In [1]:
pip freeze

absl-py==1.2.0
aiobotocore==2.3.4
aiohttp==3.8.1
aioitertools==0.10.0
aiosignal==1.2.0
alembic==1.8.1
anyio==3.6.1
appnope==0.1.3
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
astunparse==1.6.3
async-timeout==4.0.2
attrs==22.1.0
Babel==2.10.3
backcall==0.2.0
beautifulsoup4==4.11.1
black==22.6.0
bleach==5.0.1
botocore==1.24.21
cachetools==5.2.0
certifi==2022.6.15
cffi==1.15.1
charset-normalizer==2.1.0
click==8.1.3
cloudpickle==2.1.0
commonmark==0.9.1
coverage==6.4.3
cycler==0.11.0
databricks-cli==0.17.0
debugpy==1.6.2
decorator==5.1.1
defusedxml==0.7.1
docker==5.0.3
docutils==0.19
entrypoints==0.4
executing==0.9.1
fastapi==0.79.0
fastjsonschema==2.16.1
flake8==5.0.4
Flask==2.2.2
flatbuffers==1.12
fonttools==4.34.4
frozenlist==1.3.1
fsspec==2022.7.1
gast==0.4.0
gcsfs==2022.7.1
gitdb==4.0.9
GitPython==3.1.27
google-api-core==2.8.2
google-auth==2.10.0
google-auth-oauthlib==0.4.6
google-cloud-core==2.3.2
google-cloud-storage==2.5.0
google-crc32c==1.3.0
google-pasta==0.2.

In [1]:
from tensorflow.keras import utils, layers, models
# from tensorflow.keras.backend import expand_dims

ModuleNotFoundError: No module named 'tensorlfow'

## Explore Data

In [ ]:
data = json.load(open("../raw_data/annotations/instances_default.json"))

In [ ]:
data

In [ ]:
data.keys()

In [ ]:
# data['info']

In [ ]:
# data['images'][:2]

In [ ]:
# data['images'][-3:-1]

In [ ]:
# for image in data['images']:
#     if image['id']==3400 or image['id']==2166:
#         print('file name', image['file_name'])

In [ ]:
from IPython.display import Image, display

Image1 = '../raw_data/images/0001_A_H_0_A.jpg'
display(Image(Image1))
# for imageName in listOfImageNames:
#     display(Image(filename=imageName))

In [ ]:
dict_keys=['licenses', 'info', 'annotations', 'images', 'categories']

In [ ]:
type(data['annotations'])

In [ ]:
len(data['annotations'])

In [ ]:
len(data['images'])

In [ ]:
data['annotations'][10:12]

In [ ]:
data['annotations'][0]['category_id']

In [ ]:
annotations = data['annotations']

In [ ]:
for dicts in annotations:
#     print('id',dicts['id'],'image_id', dicts['image_id'],'category',dicts['category_id'])
    
    if dicts['category_id'] == 8:
        print('healthy',dicts['image_id'])

## Merge Category with image #
Found multiple categories PER image
Only 2 images had category =8 meaning 'healthy' and those are image_id 2166 & 3400 but these images have other categories as well

In [ ]:
data['categories']

In [ ]:
rows = [[dicts['id'],dicts['area'], dicts['image_id'],dicts['category_id']] for dicts in annotations]

In [ ]:
df = pd.DataFrame(rows, columns=["Annot_id","area","image_id","category"])
df

In [ ]:
df.groupby(['image_id']).size().hist()

In [ ]:
category_ = df.groupby('image_id').apply(lambda group: group['category'].unique())

In [ ]:
category_.map(sorted)
    

In [ ]:
len(df['image_id'].unique())

In [ ]:
plt.hist(df['category']);

In [ ]:
len(data['licenses'])

In [ ]:
len(data['info'])

In [ ]:
df[df['image_id']==100]

In [ ]:
images = data['images']
images

In [ ]:
image_rows = [[image['id'],image['file_name']] for image in images]

In [ ]:
image_df = pd.DataFrame(image_rows, columns = ['image_id','file_name'])
image_df

In [ ]:
merged_df = df.merge(image_df)
merged_df

In [ ]:
merged_df[merged_df['file_name']=="images/0002_G_I_45_A.jpg"]

In [ ]:
merged_df[merged_df['file_name']=="images/0002_G_I_45_A.jpg"].groupby('category').sum('area')

In [ ]:
merged_df[merged_df['file_name']=="images/0001_E_H_0_C.jpg"]

In [ ]:
merged_df[merged_df['file_name']=="images/0001_E_H_0_C.jpg"].groupby('category').sum('area')

In [ ]:
files=merged_df['file_name'].unique().tolist()
len(files)

In [ ]:
files

## Split the images into Train and Test sets

In [ ]:
import random
random.seed(2)
random.shuffle(files)

train_img = files[:int(len(files)*.70)] 

test_img= files[int(len(files)*.70):] 

In [2]:
list1=list(range(1,2691,1))
list1[:2],list1[-3:]

([1, 2], [2688, 2689, 2690])

In [3]:
train=int(len(list1)*0.7)

In [8]:
train_list = list1[:train+1]
train_list[-1]

1883

In [9]:
len(train_list)

1883

In [11]:
test_list = list1[train+1:]
test_list[-1], test_list[:2]

(2690, [1884, 1885])

In [6]:
len(test_list)

808

## Create Function to classify unhealthy and healthy lemon images

In [ ]:
def classify_category(merged_df,file_name):
    targeted=merged_df.loc[merged_df['file_name'] == file_name].groupby('category').sum('area')
    result=[]
    for idx in targeted.index:
        if idx in [2,3,4]:
            result.append(True)
        else:
            result.append(False)
    if any(result):
        return("Unhealthy_lemon")
    else:
        return("Healthy_lemon")

In [ ]:
for file in train_img:
        shutil.copy2(f"../raw_data/{file}",f"../raw_data/classified_images/Train_lemon/{classify_category(merged_df,file)}/{file.split('/')[-1]}")

In [ ]:
for file in test_img:
    shutil.copy2(f"../raw_data/{file}",f"../raw_data/classified_images/Test_lemon/{classify_category(merged_df,file)}/{file.split('/')[-1]}")

In [ ]:
# for file in files:
#     shutil.copy2(f"../raw_data/{file}",
#                  f"../raw_data/classified_images/{classify_category(merged_df,file)}/{file.split('/')[-1]}") 
    

In [ ]:
len(os.listdir("../raw_data/classified_images/Train_lemon/Healthy_lemon"))

In [ ]:
len(os.listdir("../raw_data/classified_images/Train_lemon/Unhealthy_lemon"))

In [ ]:
len(os.listdir("../raw_data/classified_images/Test_lemon/Healthy_lemon"))

In [ ]:
len(os.listdir("../raw_data/classified_images/Test_lemon/Unhealthy_lemon"))

## Make a baseline model

In [ ]:
#Use this method to quickly read a small number of images
filepath = os.listdir("..raw_data/classified_images/Train_lemon/Healthy_lemon") # a folder of some images
filepath

In [ ]:
# lemon_data = []
# for file in filepath:
#     lemon_data.append(image.imread(f'..raw_data/classified_images/Train_lemon/Healthy_lemon/{file}'))

In [ ]:
lemon_data = utils.image_dataset_from_directory('..raw_data/classified_images/Train_lemon')
#this should give you a list of how many files per category eg healthy or unhealthy

In [ ]:
# from pycocotools.coco import COCO
# coco = COCO("../raw_data/annotations/instances_default.json")

In [ ]:
# coco.anns

In [ ]:
# coco.dataset